In [ ]:
import pandas as pd
import math
import random

In [ ]:
data = pd.read_excel("/content/Lab 6 A star.xlsx")
data.head()

,Location,Latitude,Longitude,Unnamed: 3
0,DJSCE,19.107644,72.837318,NaN
1,Cooper Hospital,19.107928,72.835984,NaN
2,Mithibai,19.103075,72.837716,NaN
3,Irla Petrol Pump,19.107881,72.840121,NaN
4,Merwan Cake shop,19.120065,72.845228,NaN


In [ ]:
data = data.drop(columns = ['Unnamed: 3'])
data.head()

,Location,Latitude,Longitude
0,DJSCE,19.107644,72.837318
1,Cooper Hospital,19.107928,72.835984
2,Mithibai,19.103075,72.837716
3,Irla Petrol Pump,19.107881,72.840121
4,Merwan Cake shop,19.120065,72.845228


In [ ]:
data = data.set_index('Location')
data.head()

,Latitude,Longitude
Location,,
DJSCE,19.107644,72.837318
Cooper Hospital,19.107928,72.835984
Mithibai,19.103075,72.837716
Irla Petrol Pump,19.107881,72.840121
Merwan Cake shop,19.120065,72.845228


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, DJSCE to Pizza By the Bay
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   34 non-null     float64
 1   Longitude  34 non-null     float64
dtypes: float64(2)
memory usage: 816.0+ bytes


In [ ]:
df = data.copy()

In [ ]:
def getloc(target_location):
  global df
  location_info = df.loc[target_location]
  latitude = location_info['Latitude']
  longitude = location_info['Longitude']
  return latitude, longitude

In [ ]:
start = 'DJSCE'
getloc(start)

(19.1076444256361, 72.837317642686)

In [ ]:
end = 'Trident'
getloc(end)

(18.9277246936148, 72.8211978452959)

In [ ]:
def haversine(start, end):
  lat1 = getloc(start)[0]
  lon1 = getloc(start)[1]
  lat2 = getloc(end)[0]
  lon2 = getloc(end)[1]

  dLat = (lat2 - lat1) * math.pi / 180.0
  dLon = (lon2 - lon1) * math.pi / 180.0

  lat1 = (lat1) * math.pi / 180.0
  lat2 = (lat2) * math.pi / 180.0
  a = (pow(math.sin(dLat / 2), 2) + pow(math.sin(dLon / 2), 2) * math.cos(lat1) * math.cos(lat2))
  rad = 6371
  c = 2 * math.asin(math.sqrt(a))
  return rad * c

In [ ]:
haversine(start, end)

20.077803077601043

In [ ]:
def calch(end):
  global df
  df['h'] = None
  for index, row in df.iterrows():
    df.at[index, 'h'] = haversine(index, end)
  df.head()

In [ ]:
def connections():
  global df
  df['Movegen'] = None
  for location, row in df.iterrows():
    num_to_remove = random.randint(5, 8)
    neighbours = df.index.tolist()
    neighbours.remove(location)
    elements_to_remove = random.sample(neighbours, num_to_remove)
    for element in elements_to_remove:
        neighbours.remove(element)
    df.at[location, 'Movegen'] = neighbours
  df.head()

In [ ]:
def ReconstructPath(N):
  global df
  path = [N]
  while True:
    parent = df.at[N, 'parent']
    if parent != None:
      path.append(parent)
      N = parent
    else:
      break
  return path

In [ ]:
def lowf(open):
  global df
  minn = 9999
  for node in open:
    if df.at[node, 'f']<minn:
      minn = df.at[node, 'f']
      final = node
  return final

In [ ]:
def A_(S, G):
  global df
  df['g'] = [999]*len(df)
  df['parent'] = None
  df.at[S, 'parent'] = None
  df.at[S, 'g'] = 0
  df['f'] = None
  df.at[S, 'f'] = df.at[S, 'g'] + df.at[S, 'h']
  open = [S]
  closed = []
  while open != []:
    N = lowf(open)
    closed.append(N)
    open.remove(N)
    if G == N:
      return ReconstructPath(N)
    for M in df.at[N, 'Movegen']:
      if (df.at[N, 'g']+haversine(N, M))<df.at[M, 'g']:
        df.at[M, 'parent'] = N
        df.at[M, 'g'] = df.at[N, 'g'] + haversine(N, M)
        df.at[M, 'f'] = df.at[M, 'g'] + df.at[M, 'h']
        if M in open:
          continue
        if M in closed:
          propagate_Improvement(M, closed)
        else:
          open.append(M)
  return []

In [ ]:
def propagate_Improvement(M, closed):
  global df
  for X in df.at[M, 'Movegen']:
    if (df.at[M, 'g']+haversine(M, X))<df.at[X, 'g']:
      df.at[X, 'parent'] = M
      df.at[X, 'g']=df.at[M, 'g']+haversine(M, X)
      df.at[X, 'f'] = df.at[X, 'g'] + df.at[X, 'h']
      if X in closed:
        propagate_Improvement(X, closed)

In [ ]:
start = 'DJSCE'
end = 'Trident'
connections()
calch(end)
A_(start, end)

['Trident', 'DJSCE']

In [ ]:
df

,Latitude,Longitude,Movegen,h,g,parent,f
Location,,,,,,,
DJSCE,19.107644,72.837318,"[Cooper Hospital, Irla Petrol Pump, Merwan Cak...",20.077803,0.000000,None,20.077803
Cooper Hospital,19.107928,72.835984,"[DJSCE, Irla Petrol Pump, Merwan Cake shop, 4 ...",20.097899,0.143626,DJSCE,20.241525
Mithibai,19.103075,72.837716,"[DJSCE, Cooper Hospital, Irla Petrol Pump, Mer...",19.575197,999.000000,None,None
Irla Petrol Pump,19.107881,72.840121,"[DJSCE, Cooper Hospital, Mithibai, Merwan Cake...",20.131048,0.295675,DJSCE,20.426723
Merwan Cake shop,19.120065,72.845228,"[DJSCE, Mithibai, Irla Petrol Pump, Juhu Circl...",21.535952,1.611895,DJSCE,23.147847
Juhu Circle,19.116195,72.829870,"[DJSCE, Cooper Hospital, Mithibai, 4 bunglow, ...",20.976795,999.000000,None,None
4 bunglow,19.123265,72.827806,"[DJSCE, Cooper Hospital, Irla Petrol Pump, Juh...",21.754201,2.003913,DJSCE,23.758114
Kokilaben Hospital,19.131352,72.825063,"[DJSCE, Irla Petrol Pump, Merwan Cake shop, 4 ...",22.645942,2.933744,DJSCE,25.579685
Lokhandwala Market,19.145934,72.822943,"[Mithibai, Irla Petrol Pump, Merwan Cake shop,...",24.264445,4.517475,DJSCE,28.78192


In [ ]:
haversine('DJSCE', 'Heera Panna')+ haversine('Heera Panna', 'Trident')

20.358269304280867

In [ ]:
haversine('DJSCE', 'Trident')

20.077803077601043

In [ ]:
data.head()

,Latitude,Longitude
Location,,
DJSCE,19.107644,72.837318
Cooper Hospital,19.107928,72.835984
Mithibai,19.103075,72.837716
Irla Petrol Pump,19.107881,72.840121
Merwan Cake shop,19.120065,72.845228


In [ ]:
df = data.copy()
start = 'Juhu Beach'
end = 'Gateway of India'
connections()
calch(end)
A_(start, end)

['Gateway of India', 'Juhu Beach']

In [ ]:
df

,Latitude,Longitude,Movegen,h,g,parent,f
Location,,,,,,,
DJSCE,19.107644,72.837318,"[Cooper Hospital, Mithibai, Irla Petrol Pump, ...",20.62603,1.506176,Juhu Beach,22.132206
Cooper Hospital,19.107928,72.835984,"[DJSCE, Mithibai, Irla Petrol Pump, Merwan Cak...",20.656162,1.424419,Juhu Beach,22.080581
Mithibai,19.103075,72.837716,"[DJSCE, Irla Petrol Pump, Merwan Cake shop, Ju...",20.118559,1.278164,Juhu Beach,21.396723
Irla Petrol Pump,19.107881,72.840121,"[DJSCE, Mithibai, Juhu Circle, 4 bunglow, Koki...",20.658435,1.756011,Juhu Beach,22.414447
Merwan Cake shop,19.120065,72.845228,"[DJSCE, Cooper Hospital, Mithibai, Irla Petrol...",22.033188,3.077029,Juhu Beach,25.110217
Juhu Circle,19.116195,72.829870,"[DJSCE, Mithibai, Irla Petrol Pump, Merwan Cak...",21.580934,1.960205,Juhu Beach,23.541139
4 bunglow,19.123265,72.827806,"[DJSCE, Cooper Hospital, Mithibai, Irla Petrol...",22.372816,2.716043,Juhu Beach,25.08886
Kokilaben Hospital,19.131352,72.825063,"[DJSCE, Cooper Hospital, Mithibai, Irla Petrol...",23.282298,3.613864,Juhu Beach,26.896163
Lokhandwala Market,19.145934,72.822943,"[Cooper Hospital, Mithibai, Irla Petrol Pump, ...",24.912366,5.245150,Juhu Beach,30.157515


In [ ]:
df = data.copy()
start = 'Juhu Beach'
end = 'Gateway of India'
connections()
print(df.at[start, 'Movegen'])
df.at[start, 'Movegen'].remove(end)
print(df.at[start, 'Movegen'])

['Mithibai', 'Irla Petrol Pump', 'Merwan Cake shop', 'Juhu Circle', '4 bunglow', 'Kokilaben Hospital', 'Lokhandwala Market', 'Sea Princess Hotel', 'Novotel', 'Nanavati Hospital', 'Milan Subway', 'Siddhivinayak', 'Haji Ali', 'Jaslok ', 'Bade Miya', 'Gateway of India', 'Saifi', 'Tarapor Aquarium', 'Malabar Hills', 'Charni rd Station', 'Girgaon chowpaty', 'Wankhede Stadium', 'Trident', 'Churchgate Station', 'Pizza By the Bay']
['Mithibai', 'Irla Petrol Pump', 'Merwan Cake shop', 'Juhu Circle', '4 bunglow', 'Kokilaben Hospital', 'Lokhandwala Market', 'Sea Princess Hotel', 'Novotel', 'Nanavati Hospital', 'Milan Subway', 'Siddhivinayak', 'Haji Ali', 'Jaslok ', 'Bade Miya', 'Saifi', 'Tarapor Aquarium', 'Malabar Hills', 'Charni rd Station', 'Girgaon chowpaty', 'Wankhede Stadium', 'Trident', 'Churchgate Station', 'Pizza By the Bay']


In [ ]:
df = data.copy()
start = 'Juhu Beach'
end = 'Gateway of India'
connections()
df.at[start, 'Movegen'].remove(end)
calch(end)
A_(start, end)

['Gateway of India', 'Siddhivinayak', 'Juhu Beach']

In [ ]:
df

,Latitude,Longitude,Movegen,h,g,parent,f
Location,,,,,,,
DJSCE,19.107644,72.837318,"[Cooper Hospital, Mithibai, Merwan Cake shop, ...",20.62603,1.506176,Juhu Beach,22.132206
Cooper Hospital,19.107928,72.835984,"[DJSCE, Mithibai, Irla Petrol Pump, Merwan Cak...",20.656162,1.424419,Juhu Beach,22.080581
Mithibai,19.103075,72.837716,"[Irla Petrol Pump, Merwan Cake shop, Juhu Circ...",20.118559,1.278164,Juhu Beach,21.396723
Irla Petrol Pump,19.107881,72.840121,"[Mithibai, Merwan Cake shop, Juhu Circle, 4 bu...",20.658435,19.276224,Siddhivinayak,39.934659
Merwan Cake shop,19.120065,72.845228,"[DJSCE, Mithibai, Irla Petrol Pump, Juhu Circl...",22.033188,3.077029,Juhu Beach,25.110217
Juhu Circle,19.116195,72.829870,"[DJSCE, Cooper Hospital, Mithibai, Irla Petrol...",21.580934,1.960205,Juhu Beach,23.541139
4 bunglow,19.123265,72.827806,"[DJSCE, Irla Petrol Pump, Merwan Cake shop, Ju...",22.372816,20.938432,Siddhivinayak,43.311248
Kokilaben Hospital,19.131352,72.825063,"[DJSCE, Cooper Hospital, Mithibai, Irla Petrol...",23.282298,3.613864,Juhu Beach,26.896163
Lokhandwala Market,19.145934,72.822943,"[DJSCE, Cooper Hospital, Mithibai, Irla Petrol...",24.912366,5.245150,Juhu Beach,30.157515


In [ ]:
# reason : given an three points, they either form a triangle or lie in a straight line

# as for lying in a straight line, a+b=c
# as for lying in a triangle, a+b > c

# but in the code we only check for less than values (a+b<c)



# ALso the graph we have is a fully connected one #### Took care of this ####
# Also we are using haversine distance for both g & h



#### Remove the end node from the Movegen of start